In [1]:
import os, sys, subprocess
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
""" Function to convert sector to polar angle """
def sector2polar(sector):
    polarAngle = 270.0 - sector # Returns in the range [-90, +270] degrees rather than [-180, +180]
    if polarAngle >=180.0 and polarAngle <= 270.0:
        polarAngle = polarAngle - 360.0
    return polarAngle

""" Compute axial velocity """
def axialVel(U, V, sector):
    polarAngle = sector2polar(sector)
    fDir = np.deg2rad(polarAngle)
    #print 'polar angle: ', polarAngle, 'deg = ', fDir, ' radians'
    Uax = U*np.cos(fDir) + V*np.sin(fDir)
    #print U, V, Uax
    return Uax

In [3]:
""" Function to create time series plot of a variable  """
def plotTimeSeriesSingle(figLoc, fileName, labelText, time, var, lineStyle='-', markersize = 2, lineWidth=1):
    plt.figure(1)
    plt.plot(time, var, lineStyle, linewidth = lineWidth, markersize = markersize)
    plt.xlabel('Time (s)')
    plt.ylabel(labelText)
    if not os.path.exists(figLoc):
        os.makedirs(figLoc)
    figName =  os.path.join(figLoc, fileName)
    plt.savefig(figName)
    plt.close()

In [4]:
"""" Function to gather probe names from the probe location """
def gatherProbeNames(probeLoc):
    probeNames = [pname for pname in os.listdir(probeLoc) if os.path.isdir(os.path.join(probeLoc, pname))]
    if 'surfaces' in probeNames:
        probeNames.remove('surfaces')
    if 'lines' in probeNames:
        probeNames.remove('lines')
    probeNames.sort()
    return probeNames

In [5]:
""" Define the function to read probe data """
def processRotorProbeData(skipTime, Timet, Ut, Vt, Wt, kt):
    U    = []
    V    = []
    W    = []

    for i in range(len(Timet)):
        U.append(float(Ut[i].split('(')[1]))
        V.append(float(Vt[i]))
        W.append(float(Wt[i].split(')')[0]))

    Time = np.array(Timet)
    tStart = float(Timet[0])
    indTstat = np.argmax((Time - tStart) > float(skipTime))
    indTEnd =  len(Timet)

    return Time[indTstat:indTEnd], np.array(U[indTstat:indTEnd]), np.array(V[indTstat:indTEnd]), np.array(W[indTstat:indTEnd]), np.array(kt[indTstat:indTEnd])

In [6]:
""" Function to load probe data for a probe """
def loadProbeData(probeLoc, probeName, startTime, skipTime):
    probeFileU = os.path.join( probeLoc, probeName, str(startTime), 'U')
    probeFilek = os.path.join( probeLoc, probeName, str(startTime), 'k')

    probes  = pd.read_csv(probeFileU,sep='\s+',skiprows=3,
            names=['Time','uh','vh','wh'])
    probesk = pd.read_csv(probeFilek,sep='\s+',skiprows=3,
            names=['Time','kh'])

    # Read the probe data
    timeHub, UHub, VHub, WHub, kHub = processRotorProbeData (
            skipTime, probes['Time'], probes['uh'], probes['vh'], probes['wh'], probesk['kh'])
    
    return timeHub, UHub, VHub, WHub, kHub

In [ ]:
# Main code
simList = ['kOmega_des_asl_coarseMesh', \
           'kOmega_iddes_asl_coarseMesh', \
           'kOmega_des_asl_betaStar0.03', \
           'kOmega_iddes_asl_betaStar0.03']
sector = 210.0
startTime = 0
skipTime = 0.0

In [ ]:
for sim in simList:
    case_name = 'ask_' + sim
    print ('case_name: ', case_name)
    probeLoc = os.path.join(case_name, 'probes')
    probeNames = gatherProbeNames(probeLoc)
    #print 'probeNames:\n', probeNames
    
    for probe in probeNames:
        time, U, V, W, k = loadProbeData(probeLoc, probe, startTime, skipTime)
        Uax = axialVel (U, V, sector)
        figLoc = os.path.join('plots','probes', sim)
        plotTimeSeriesSingle(figLoc, probe+'_U.png', 'X-component of velocity, U (m/s)', time, U)
        plotTimeSeriesSingle(figLoc, probe+'_V.png', 'Y-component of velocity, V (m/s)', time, V)
        plotTimeSeriesSingle(figLoc, probe+'_W.png', 'Z-component of velocity, W (m/s)', time, W)
        plotTimeSeriesSingle(figLoc, probe+'_Uax.png', 'Axial wind velocity, Uax (m/s)', time, Uax)
        plotTimeSeriesSingle(figLoc, probe+'_TKE.png', 'Turbulent kinetic energy (m^2/s^2)', time, k)
        #plotTimeSeriesSingle(figLoc, fileName, labelText, time, var, lineStyle='-', markersize = 2, lineWidth=1):


case_name:  ask_kOmega_des_asl_coarseMesh
case_name:  ask_kOmega_iddes_asl_coarseMesh
case_name:  ask_kOmega_des_asl_betaStar0.03
